## EDA on students test data

We are given with 5 collections from a MongoDB database
Each collection details are given below.

1. attempts: It holds summary data of student's attempt. Each test by a student becomes an attempt. If student takes 5 tests (same or different test), it will create 5 attempts. If student abandones the test, isAbandon flag is set to true. We consider only isAbandon = false documents. 

2. attemptDetails: Question level details such as time taken on each question can be found in this collection. Each attempt has a corresponding attemptDetails.

3. practicesets: List of tests. Array 'questions' holds list of questions. Test can be created organically or can be created from the questions of question pool.

4. questions: list of questions. One question can be part of many tests. Question can be put in the pool using flag 'isAllowReuse'

5. grades/subjects/topics - master data of exam, related subjects and topics

6. users: list of students including teachers and parents

## Objective:
1. Does more practice mean better performance?
2. Detect fraud done by some test takers. Fraud can be based on
    - Number of consecutive correct answers
    - Time spent on each questions when answer is correct
    - Time spent outside the test window (offscreen time)
    - Time spent by other users of similar profile on this question
    - Strength and weakness of the student on the topic
    - Time spent by the student on similar questions
3. Student clusters based on different attributes e.g. performance, time spent, consistency, participation in discussion etc
4. A teacher may mark a question easy/medium/hard (Perceived hardness) but we want to give feedback to teachers on what's real hardness based on student's data (how many students answered correctly, how much time taken, whether student was good in the area/topic etc. )
5. Predict score in a test and challenge student to beat it 

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from pymongo import mongo_client
from pymongo import MongoClient
from pandas.io import json

In [2]:
client = MongoClient('localhost', 27017)

In [3]:
# lets see the database names
client.list_database_names()

['admin', 'config', 'internshalla', 'local']

In [4]:
db = client.internshalla

In [5]:
# lets see the total collection names present in the internshalla database
collect_names = db.list_collection_names()

In [6]:
users_df = json.json_normalize(list(db.users.find()))
attempts_df = json.json_normalize(list(db.attempts.find()))
grades_df = json.json_normalize(list(db.grades.find()))
practicesets_df = json.json_normalize(list(db.practicesets.find()))
topics_df = json.json_normalize(list(db.topics.find()))
subjects_df = json.json_normalize(list(db.subjects.find()))
questions_df = json.json_normalize(list(db.questions.find()))
attemptdetails_df = json.json_normalize(list(db.attemptdetails.find()))

In [8]:
attempts_df.head()

,__v,_id,attemptType,attemptdetails,createdAt,createdBy.name,createdBy.user,email,idOffline,isAbandoned,...,totalCorrects,totalErrors,totalMark,totalMarkeds,totalMissed,totalQuestions,totalTime,updatedAt,user,userId
0,0,5b851ffdb0a5d1ad413d8301,NaN,5b851ffdb0a5d17a103d8302,2018-08-28 09:23:57.839,teacherlpu,5b3e0266a14b7e000d9cc040,None,None,False,...,35,20,35.0,0,0,55,2887958,2018-08-28 10:12:25.489,5b82922edd6c9c6e6db00fb0,NaN
1,0,5b852003b0a5d136aa3d8371,NaN,5b852003b0a5d190863d8372,2018-08-28 09:17:11.950,teacherlpu,5b3e0266a14b7e000d9cc040,None,None,False,...,31,24,31.0,0,0,55,3300006,2018-08-28 10:12:39.239,5b829227dd6c9c4c7fb00da0,NaN
2,0,5b851ff7dd6c9c8cedb051d3,NaN,5b851ff7dd6c9c400eb051d4,2018-08-28 09:17:00.084,teacherlpu,5b3e0266a14b7e000d9cc040,None,None,False,...,25,9,25.0,0,21,55,3299999,2018-08-28 10:12:21.337,5b82922cdd6c9c2801b00f20,NaN
3,0,5b852014b0a5d10fc33d83e4,NaN,5b852014b0a5d11fd13d83e5,2018-08-28 09:17:28.725,teacherlpu,5b3e0266a14b7e000d9cc040,None,None,False,...,29,15,29.0,0,11,55,3300049,2018-08-28 10:12:59.708,5b82922add6c9c5f61b00e6a,NaN
4,0,5b85200948598b0aa6eabf57,NaN,5b85200948598b0d5beabf58,2018-08-28 09:17:16.906,teacherlpu,5b3e0266a14b7e000d9cc040,None,None,False,...,16,19,16.0,0,20,55,3300005,2018-08-28 10:12:38.440,5b82922edd6c9cd998b00f86,NaN


In [13]:
attemptdetails_df.head()

,QA,__v,_id,attempt,createdAt,isAbandoned,practicesetId,updatedAt,user
0,"[{'question': 5b82ecb0dd6c9c00e1b01857, 'timeE...",0,5b83d566af15dcb2402312df,5b83d566af15dc6d322312de,2018-08-27 10:41:42.351,False,5b82ecb0dd6c9c3785b01856,2018-08-27 10:41:42.351,5b82922cdd6c9c67f5b00ef6
1,"[{'question': 5b82ecb0dd6c9c00e1b01857, 'timeE...",0,5b83d5723a227f518ce1770b,5b83d5723a227f350be1770a,2018-08-27 10:41:54.195,False,5b82ecb0dd6c9c3785b01856,2018-08-27 10:41:54.195,5b82922edd6c9c0c57b00f90
2,"[{'question': 5b82ecb0dd6c9c00e1b01857, 'timeE...",0,5b83d57edd6c9c2b7db02a8f,5b83d57edd6c9c8ec5b02a8e,2018-08-27 10:42:06.566,False,5b82ecb0dd6c9c3785b01856,2018-08-27 10:42:06.566,5b82922edd6c9c52c9b00fbe
3,"[{'question': 5b82ecb0dd6c9c00e1b01857, 'timeE...",0,5b83d58648598b77a0ea8a2a,5b83d58548598bb42dea8a29,2018-08-27 10:42:14.080,False,5b82ecb0dd6c9c3785b01856,2018-08-27 10:42:14.080,5b82922edd6c9c55fbb00f92
4,"[{'question': 5b82ecb0dd6c9c00e1b01857, 'timeE...",0,5b83d58c48598b0d31ea8a8c,5b83d58c48598bec32ea8a8b,2018-08-27 10:42:20.879,False,5b82ecb0dd6c9c3785b01856,2018-08-27 10:42:20.879,5b829227dd6c9c9dacb00da2


In [7]:
users_df.head()

,__v,_id,about,avatar._id,avatar.fileName,avatar.fileUrl,avatar.mimeType,avatar.path,avatar.size,avatarMD._id,...,state,status,streamUrl,street,studentExclusive,theme,trainingCertifications,updatedAt,userId,web
0,1,5ba5dd1b7fd1c520c12710ee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,True,,NaN,False,NaN,NaN,2018-09-22 06:11:39.601,s6o9mreasdhzdufo@gmail.com,NaN
1,2,5ba5dd1d7fd1c5a3fb2710f2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Punjab,True,,NaN,False,NaN,NaN,2018-09-22 06:11:41.955,d8o2oank.iamna@gmail.com,NaN
2,2,5ba5dd217fd1c56a6e2710f8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Punjab,True,,NaN,False,NaN,NaN,2018-09-22 06:11:45.281,9591871089,NaN
3,2,5ba5dd237fd1c5babe2710fd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Punjab,True,,NaN,False,NaN,NaN,2018-09-22 06:11:47.159,9819563408,NaN
4,1,5ba5de707fd1c552b32712cd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,True,,NaN,False,NaN,NaN,2018-09-22 06:17:20.076,9035735367,NaN


In [9]:
grades_df.head()

,ShortName,__v,_id,active,adaptive,countryCode,createdAt,empIndex,isDefault,levels,moveType,name,slugfly,subjects,updatedAt,view
0,ECE,0.0,6cd9a31607654fe0b5db94b9,True,False,IN,2017-10-09 06:09:57.000,NaN,False,NaN,2.0,Electronics & Communication Engineering,electronics--communication-engineering,"[0e8b97135a8e451a917644e8, d9208d94105e43c3ad7...",2017-10-09 06:09:57.000,NaN
1,MCH,0.0,583b478ea0399b0001798fbb,True,False,IN,2016-11-27 20:52:30.100,NaN,False,NaN,2.0,Mechanical Engineering,mechanical-engineering,"[583b47caa0399b0001798fbc, 583b47dda0399b00017...",2017-10-09 02:18:43.804,banking.html
2,PRT,0.0,b0e56effc8ff4d969c41c40c,True,False,IN,2017-03-30 11:14:17.000,True,True,"[{'name': 'Level 1', 'value': 1.0}, {'name': '...",2.0,Placement Readiness Tests,placement-readiness-tests,"[a262c47026a44718acd41306, e8cfb0328cda490ca3e...",2017-03-30 11:14:17.000,NaN
3,Psy,NaN,5bacca975cb8c3baf1c24301,True,False,IN,2018-09-27 12:18:32.640,NaN,True,NaN,NaN,Psychometric Test,psychometric-test,"[5bacca975cb8c3baf1c24302, 5bacca975cb8c3baf1c...",2018-09-27 12:18:32.640,NaN
4,PRG,0.0,5bc97c1e6dcc694f0247ea57,True,False,IN,2018-10-19 06:39:26.505,NaN,True,"[{'name': 'Level 1', 'value': 1.0}, {'name': '...",1.0,Programming,programming,"[3835d8c367564425b162d7db, 5bfdb373ab9c272b3ea...",2018-10-19 06:39:26.505,


In [10]:
practicesets_df.head()

,__v,_id,accessMode,attemptAllowed,classRooms,countryCode,createMode,createdAt,currencyCode,demographicData.city,...,title,titleLower,totalAttempt,totalJoinedStudent,totalQuestion,totalTime,updatedAt,user,userInfo._id,userInfo.name
0,41,5b3e0b17a14b7e000d9cc116,invitation,NaN,[],IN,webUpload,2018-07-05 12:12:07.958,INR,False,...,Novice to Ninja <JAVA> Baseline Test,novice to ninja <java> baseline test,0,0,30,5400.0,2018-10-21 16:58:33.836,5b3778f43331fd000d75e277,5b3778f43331fd000d75e277,LPU Admin
1,15,5b3e23e4e87c53000d7cb59d,invitation,0.0,"[5b48c84c1e6eaf000dca5814, 5b3e1c67a14b7e000d9...",IN,webUpload,2018-07-05 13:57:56.536,INR,False,...,Placement Readiness Test (Lite) - Baseline Test,placement readiness test (lite) - baseline test,31,31,45,3600.0,2018-07-05 14:01:21.741,5b3778f43331fd000d75e277,5b3778f43331fd000d75e277,LPU Admin
2,2,5b49c3773222c8000c196745,invitation,0.0,[5b48c84c1e6eaf000dca5814],IN,webManual,2018-07-14 09:33:43.585,INR,False,...,Placement Readiness Test <C++ Coding> Practice...,placement readiness test <c++ coding> practice...,5,2,2,900.0,2018-07-14 09:38:42.221,5b3778f43331fd000d75e277,5b3778f43331fd000d75e277,LPU Admin
3,2,5b49c5853222c8000c1967bc,invitation,0.0,[5b48c84c1e6eaf000dca5814],IN,webManual,2018-07-14 09:42:29.024,INR,False,...,Placement Readiness Test <Java Coding> - Pract...,placement readiness test <java coding> - pract...,7,6,2,900.0,2018-07-14 09:45:31.295,5b3778f43331fd000d75e277,5b3778f43331fd000d75e277,LPU Admin
4,0,5b49c2ed3222c8000c1966cb,invitation,0.0,[5b48c84c1e6eaf000dca5814],IN,webUpload,2018-07-14 09:31:25.634,INR,False,...,KSOLVES– Java – Practice Set – 1-QA,ksolves– java – practice set – 1-qa,28,28,20,NaN,2018-07-14 09:31:25.634,5b3778f43331fd000d75e277,5b3778f43331fd000d75e277,LPU Admin


In [11]:
topics_df.head()

,__v,_id,active,createdAt,name,slugfly,subject,updatedAt
0,0.0,0e2e57dd349244c19b819f69,True,2017-08-31T10:39:24,Laplace Transform,laplace-transform,9c6bdbbdab294f3588ed7bb4,2017-08-31T10:39:24
1,0.0,33793df021c84600955ca6a0,True,2017-08-31T10:39:24,Miscellaneous,miscellaneous,9c6bdbbdab294f3588ed7bb4,2017-08-31T10:39:24
2,0.0,42fda27216e543ed8c5877c0,True,2017-08-31T10:39:24,Probability & Statistics Fundamentals,probability--statistics-fundamentals,9c6bdbbdab294f3588ed7bb4,2017-08-31T10:39:24
3,0.0,df3258f2056e4e73b402bb8a,True,2017-08-31T10:39:24,Solid Waste Management,solid-waste-management,3189e97498e640cd8417bf14,2017-08-31T10:39:24
4,0.0,997795716b43449d9928453d,True,2017-08-31T10:39:24,Dimensional Analysis,dimensional-analysis,7cf7bdb1c1464e82b21de6c9,2017-08-31T10:39:24


In [12]:
subjects_df.head()

,__v,_id,active,countryCode,createdAt,grade,levels,name,slugfly,status,topics,updatedAt
0,0.0,74296ce7aa304946b5629635,True,IN,2017-08-31 10:39:24.000,d51036e3049840aeab2bb394,"[{'name': 'Level 1', 'value': 1.0, 'quantity':...",Concrete Structures (Rcc),concrete-structures-rcc,NaN,"[01b7f0900c4c4bcab93ad326, 547f8dda522343aaa6e...",2017-08-31 10:39:24.000
1,0.0,5ab2629e792f9e000be365d4,True,IN,2018-03-21 13:48:14.212,d51036e3049840aeab2bb394,"[{'name': 'Level 1', 'value': 1.0, 'quantity':...",Ground Improvement Engineering,ground-improvement-engineering,NaN,"[5ab262c2792f9e000be365d5, 5ab262d5792f9e000be...",2018-03-21 13:48:14.212
2,0.0,5ab098cd0f7749000ba1c331,True,IN,2018-03-20 05:14:53.218,d51036e3049840aeab2bb394,"[{'name': 'Level 1', 'value': 1.0, 'quantity':...",Construction Methods and Equipments,construction-methods-and-equipments,NaN,"[5ab098ee0f7749000ba1c332, 5ab099230f7749000ba...",2018-03-20 05:14:53.218
3,0.0,bfee82a9885e4965841fa7ee,True,IN,2017-08-31 10:39:24.000,d51036e3049840aeab2bb394,"[{'name': 'Level 1', 'value': 1.0, 'quantity':...",Hydrology,hydrology,NaN,"[9b10f05ef3074e3e806c281c, add1c673ff8445a5a99...",2017-08-31 10:39:24.000
4,0.0,5ac5e672bb5152000d09e0c3,True,IN,2018-04-05 09:03:46.149,d51036e3049840aeab2bb394,"[{'name': 'Level 1', 'value': 1.0, 'quantity':...",Project Implementation & Practices,project-implementation-and-practices,NaN,"[5ac5e68abb5152000d09e0c4, 5ac5e699bb5152000d0...",2018-04-05 09:03:46.149


In [14]:
questions_df.head()

,__v,_id,alternativeExplanations,answerExplain,answerExplainArr,answerNumber,answers,approveStatus,argumentDescription,category,...,questionType,subject._id,subject.name,tags,topic._id,topic.name,updatedAt,user,userInputDescription,wordLimit
0,1.0,5b3e0b17a14b7e000d9cc12b,[],,[],4.0,[],,,mcq,...,single,3835d8c367564425b162d7db,Core Programming,[],5a85811318fa67000d079d4c,Conditions,2018-10-21 16:53:09.339,5b3e07fea14b7e000d9cc109,,1.0
1,1.0,5b3e0b17a14b7e000d9cc117,[],,[],4.0,[],,,mcq,...,single,3835d8c367564425b162d7db,Core Programming,[],fa666afaf2114b90ac805a07,Basic Concepts,2018-12-29 17:28:21.624,5b3e07fea14b7e000d9cc109,,1.0
2,1.0,5b3e0b18a14b7e000d9cc144,[],,[],4.0,[],,,mcq,...,single,3835d8c367564425b162d7db,Core Programming,[],5b1127da4ecf13000ccf08cf,Arrays,2018-10-21 16:54:16.498,5b3e07fea14b7e000d9cc109,,1.0
3,1.0,5b3e0b18a14b7e000d9cc135,[],,[],4.0,[],,,mcq,...,single,3835d8c367564425b162d7db,Core Programming,[],5b29f5104ef218000da48ca6,Constructor and Destructor,2018-10-21 16:53:40.107,5b3e07fea14b7e000d9cc109,,1.0
4,1.0,5b3e0b17a14b7e000d9cc11c,[],,[],4.0,[],,,mcq,...,single,3835d8c367564425b162d7db,Core Programming,[],fa666afaf2114b90ac805a07,Basic Concepts,2018-10-21 16:51:46.762,5b3e07fea14b7e000d9cc109,,1.0


In [16]:
topics_df.head()

,__v,_id,active,createdAt,name,slugfly,subject,updatedAt
0,0.0,0e2e57dd349244c19b819f69,True,2017-08-31T10:39:24,Laplace Transform,laplace-transform,9c6bdbbdab294f3588ed7bb4,2017-08-31T10:39:24
1,0.0,33793df021c84600955ca6a0,True,2017-08-31T10:39:24,Miscellaneous,miscellaneous,9c6bdbbdab294f3588ed7bb4,2017-08-31T10:39:24
2,0.0,42fda27216e543ed8c5877c0,True,2017-08-31T10:39:24,Probability & Statistics Fundamentals,probability--statistics-fundamentals,9c6bdbbdab294f3588ed7bb4,2017-08-31T10:39:24
3,0.0,df3258f2056e4e73b402bb8a,True,2017-08-31T10:39:24,Solid Waste Management,solid-waste-management,3189e97498e640cd8417bf14,2017-08-31T10:39:24
4,0.0,997795716b43449d9928453d,True,2017-08-31T10:39:24,Dimensional Analysis,dimensional-analysis,7cf7bdb1c1464e82b21de6c9,2017-08-31T10:39:24
